# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import csv
from ast import literal_eval

#  &#187; METACRITIC DATASET  &#171;

In [30]:
metacritic_df = pd.read_csv("metacritic.csv")
metacritic_df = metacritic_df[['game_title', 'genre', 'platforms', 'developers',
       'release_year', 'release_month', 'release_day',
       'average_metascore_ratings']]

#Convert columns with string values to lists
for column in ['genre', 'platforms', 'developers']:
    metacritic_df[column] = metacritic_df[column].apply(literal_eval)


#Preview of first 5 rows
metacritic_df.head(5)

,game_title,genre,platforms,developers,release_year,release_month,release_day,average_metascore_ratings
0,Journey,"[Action, Platformer, 3D]","[PlayStation 4, PC, PlayStation 3]",[Tricky Pixels],2015,7,21,92.00
1,Celeste,"[Action, Platformer, 2D]","[PlayStation 4, iPhone/iPad, PC, Switch, Xbox ...",[Matt Makes Games Inc.],2018,1,25,91.25
2,INSIDE,"[Action, Platformer, 2D]","[PlayStation 4, iPhone/iPad, PC, Switch, Xbox ...",[PLAYDEAD],2016,8,23,90.50
3,Overwatch,"[Tactical, Shooter, Action, First-Person]","[PlayStation 4, PC, Xbox One]",[Blizzard Entertainment],2016,5,23,90.67
4,Shovel Knight,"[Action, Platformer, 2D]","[PlayStation 4, 3DS, PC, PlayStation 3, PlaySt...",[Yacht Club Games],2015,4,21,88.22


#  &#187; USER INPUT ANALYSIS &#171;

<strong>Idea:</strong> A user would input three video game titles as basis for the recommendation engine 

## Example of user input

In [258]:
user_input = ['Dark Souls III','League of Legends', 'The Elder Scrolls V: Skyrim Special Edition']

# Grabs rows from the metacritic data frame based on the user's inputted games
def user_game_data(df,titles):
    data_frame = df[df["game_title"].isin(titles)]
    data_frame.index = range(len(data_frame.index))
    return data_frame

#Data frame generated from grabbing data
user_df = user_game_data(metacritic_df, user_input)
user_df

,game_title,genre,platforms,developers,release_year,release_month,release_day,average_metascore_ratings
0,Dark Souls III,"[Action RPG, Role-Playing]","[PlayStation 4, PC, Xbox One]",[From Software],2016,4,12,88.33
1,The Elder Scrolls V: Skyrim Special Edition,"[Western-Style, Role-Playing]","[PlayStation 4, PC, Switch, Xbox One]",[Bethesda Game Studios],2016,10,28,79.00
2,League of Legends,"[Role-Playing, MOBA, Real-Time, Strategy, Acti...",[PC],[Riot Games],2009,10,27,78.00


## Analysis of user's inputted video games

#### Function for taking unique items in each column and generating a new dataframe

In [259]:
def count_items(df,title):
    dic = {}
    column = df[title]
    for row in column:
        for item in row:
            if item not in dic:
                dic[item] = 1
            else:
                dic[item] +=1
    return pd.DataFrame({title : list(dic.keys()), "count": list(dic.values())}).sort_values(by='count', ascending = False)

In [260]:
user_genres = count_items(user_df, 'genre')
user_platforms = count_items(user_df, 'platforms')
user_developers = count_items(user_df, 'developers')

In [261]:
user_genres

,genre,count
1,Role-Playing,3
0,Action RPG,2
2,Western-Style,1
3,MOBA,1
4,Real-Time,1
5,Strategy,1


## &#187; RECOMMEND SIMILAR GAMES (UNWEIGHTED) &#171;

<strong>Idea:</strong> Find games with similar genres without the account of having prominent genres, platforms, and developers of inputted video game titles

#### Function to find distance/count of similaraties for each category (genres, platforms, developers)

In [265]:
def unweighted_distances(u_df, meta_df):
    meta_df_length = len(meta_df)
    user_genres = count_items(u_df, 'genre')
    user_platforms = count_items(u_df, 'platforms')
    user_developers = count_items(u_df, 'developers')
    
    # Helper Function that counts
    def count_similar(column, category):
        items = list(category.iloc[:,0])
        array = []
        for row in column:
            count = 0
            for item in items:
                if item in row:
                    count += 1
            array.append(count)  
        return array
    
    def find_distance():
        genre_similar_counts = np.array(count_similar(list(meta_df['genre']),user_genres))
        platform_similar_counts = np.array(count_similar(list(meta_df['platforms']),user_platforms))
        developer_similar_counts = np.array(count_similar(list(meta_df['developers']),user_developers))
        distance_array = []
        
        for i in range(meta_df_length):
            sqr_diff_genres = np.square(len(user_genres) - genre_similar_counts[i])
            sqr_diff_platforms = np.square(len(user_platforms) - platform_similar_counts[i])
            sqr_diff_developers = np.square(len(user_developers) - developer_similar_counts[i])
            distance = np.sqrt(sqr_diff_genres + sqr_diff_platforms  + sqr_diff_developers)
            distance_array.append(distance)
        return distance_array

    return pd.DataFrame({'game_title' :list(meta_df['game_title']), 'distance': find_distance() }).sort_values(by='distance')

In [266]:
def recommend_unweighted(u_df, meta_df):
    suggest_df = meta_df[-meta_df["game_title"].isin(user_input)]
    suggest_df = suggest_df[suggest_df.average_metascore_ratings >= 70]
    distances_df = unweighted_distances(u_df, suggest_df)
    return distances_df[['game_title']]

In [267]:
recommend_unweighted(user_df,metacritic_df).head(10)

,game_title
626,Masters of Anima
617,SMITE
1032,Dragon Age: Inquisition - Jaws Of Hakkon
1609,Awesomenauts
643,Dragon Age: Inquisition
625,Awesomenauts Assemble!
676,Thronebreaker: The Witcher Tales
646,Divinity: Original Sin Enhanced Edition
2868,Dragon Age: Inquisition - Trespasser
663,Dark Souls Remastered
